In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

1. Lecture et fusion de tous les CSV

 1.1.Lecture de tous les CSV dans un dossier

In [ ]:
sns.set(style="whitegrid")
folder_path = "/home/20016340/PycharmProjects/machine_learning_course/projet/categories"
csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
df_list = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    temp_df = pd.read_csv(file_path)
    df_list.append(temp_df)

df_combined = pd.concat(df_list, axis=0, ignore_index=True)
print(df_combined.info())
print(df_combined.head())

2. Nettoyage et transformation (prétraitement)

2.1. Suppression d'une colonne inutile

In [ ]:
# Supprimer la colonne "Unnamed: 0"
if "Unnamed: 0" in df_combined.columns:
    df_combined = df_combined.drop(columns=["Unnamed: 0"])

2.2. Conversion et nettoyage des colonnes numériques

In [ ]:
# Nettoyage des colonnes de prix
def nettoyer_prix(x):
    try:
        # Enlève le symbole ₹, les virgules et les espaces superflus
        x = x.replace('₹', '').replace(',', '').strip()
        return float(x)
    except:
        return np.nan

df_combined['discount_price'] = df_combined['discount_price'].astype(str).apply(nettoyer_prix)
df_combined['actual_price'] = df_combined['actual_price'].astype(str).apply(nettoyer_prix)

df_combined['ratings'] = pd.to_numeric(df_combined['ratings'], errors='coerce')
df_combined['no_of_ratings'] = pd.to_numeric(df_combined['no_of_ratings'], errors='coerce')

print(df_combined[['discount_price', 'actual_price', 'ratings', 'no_of_ratings']].head())

2.3. Calcule du pourcentage de réduction

In [ ]:
# Calcule la réduction en pourcentage à partir de actual_price et discount_price
df_combined['reduction_percent'] = ((df_combined['actual_price'] - df_combined['discount_price']) / df_combined['actual_price']) * 100
print(df_combined[['actual_price', 'discount_price', 'reduction_percent']].head())

3. Analyse statistique descriptive


3.1. Affichage de statistiques descriptives

In [ ]:
# Statistiques descriptives pour les colonnes numériques importantes
print(df_combined[['actual_price', 'discount_price', 'reduction_percent', 'ratings', 'no_of_ratings']].describe())

3.2. Distribution des prix et du pourcentage de réduction

In [ ]:
sns.set(style="whitegrid")

# Distribution du prix réel (actual_price)
plt.figure(figsize=(10, 5))
sns.histplot(df_combined['actual_price'].dropna(), bins=50, kde=True)
plt.title("Distribution du Prix Réel (actual_price)")
plt.xlabel("Prix réel")
plt.ylabel("Fréquence")
plt.show()

# Distribution du pourcentage de réduction (reduction_percent)
plt.figure(figsize=(10, 5))
sns.histplot(df_combined['reduction_percent'].dropna(), bins=50, kde=True, color='coral')
plt.title("Distribution du Pourcentage de Réduction")
plt.xlabel("Pourcentage de réduction")
plt.ylabel("Fréquence")
plt.show()

4) Visualisations avancées

4.1) Relations entre variables

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='actual_price', y='discount_price', data=df_combined, alpha=0.5)
plt.title("Relation entre Prix Réel et Prix Réduit")
plt.xlabel("Prix Réel (actual_price)")
plt.ylabel("Prix Réduit (discount_price)")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x='reduction_percent', y='no_of_ratings', data=df_combined, alpha=0.5)
plt.title("Relation entre % de Réduction et Nombre d'Évaluations")
plt.xlabel("Pourcentage de Réduction")
plt.ylabel("Nombre d'Évaluations")
plt.show()

4.2) Visualisation de la répartition par catégorie

In [ ]:
plt.figure(figsize=(12, 6))
# On affiche les catégories par ordre décroissant du nombre de produits
sns.countplot(data=df_combined, x='main_category',
              order=df_combined['main_category'].value_counts().index)
plt.title("Distribution des Produits par Catégorie Principale")
plt.xlabel("Catégorie principale")
plt.ylabel("Nombre de produits")
plt.xticks(rotation=45)
plt.show()

4.3) Matrice de corrélation entre variables numériques

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df_combined[['ratings', 'no_of_ratings', 'discount_price',
                                  'actual_price', 'reduction_percent']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Matrice de Corrélation entre Variables Numériques")
plt.show()


5) Analyse additionnelle

5.1) Distribution des ratings

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df_combined['ratings'].dropna(), bins=30, kde=True, color='green')
plt.title("Distribution des Notes (ratings)")
plt.xlabel("Note")
plt.ylabel("Fréquence")
plt.show()

5.2) Analyse de la longueur des descriptions

In [ ]:
if 'description' in df_combined.columns:
    df_combined['description_length'] = df_combined['description'].apply(lambda x: len(str(x)))
    plt.figure(figsize=(10, 5))
    sns.histplot(df_combined['description_length'], bins=50, kde=True, color='purple')
    plt.title("Distribution de la Longueur des Descriptions")
    plt.xlabel("Nombre de caractères")
    plt.ylabel("Fréquence")
    plt.show()